In [21]:
import os 

In [22]:
%pwd

'/Users/himanshuagarwal/Desktop/Text-Summarizer'

In [23]:
%cd /users/himanshuagarwal/Desktop/Text-Summarizer

/Users/himanshuagarwal/Desktop/Text-Summarizer


/opt/anaconda3/envs/textS/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [24]:
%pwd

'/Users/himanshuagarwal/Desktop/Text-Summarizer'

In [ ]:
import torch
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [26]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=int(params.num_train_epochs),
            warmup_steps=int(params.warmup_steps),
            per_device_train_batch_size=int(params.per_device_train_batch_size),
            weight_decay=float(params.weight_decay),
            logging_steps=int(params.logging_steps),
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=int(params.eval_steps),
            save_steps=int(params.save_steps),  # CONVERT TO INT
            gradient_accumulation_steps=int(params.gradient_accumulation_steps)
        )
        return model_trainer_config

In [28]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        
        # Truncation settings
        max_input_length = 512
        max_output_length = 128
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        def truncate_example(example):
            return {
                'input_ids': example['input_ids'][:max_input_length],
                'attention_mask': example['attention_mask'][:max_input_length],
                'labels': example['labels'][:max_output_length]
            }
        
        train_dataset = dataset_samsum_pt["train"].select(range(50)).map(truncate_example)
        eval_dataset = dataset_samsum_pt["validation"].select(range(20)).map(truncate_example)

        seq2seq_data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model_pegasus,
            padding='longest'  # Efficient dynamic padding
        )

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            gradient_checkpointing=False,  # Disabled for MPS
            fp16=False  # Enable mixed precision
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )
        
        trainer.train()
        
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
        


In [30]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-08-18 11:15:54,519: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-18 11:15:54,522: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-18 11:15:54,522: INFO: common: created directory at: artifacts]
[2025-08-18 11:15:54,523: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/7h/q9ssh4wj7r3d99nqq2qzhkb80000gn/T/ipykernel_83925/1228116770.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [32]:
import os
import torch
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=int(params.num_train_epochs),
            warmup_steps=int(params.warmup_steps),
            per_device_train_batch_size=int(params.per_device_train_batch_size),
            weight_decay=float(params.weight_decay),
            logging_steps=int(params.logging_steps),
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=int(params.eval_steps),
            save_steps=int(params.save_steps),
            gradient_accumulation_steps=int(params.gradient_accumulation_steps)
        )
        return model_trainer_config

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Device detection with clear error handling
        if torch.backends.mps.is_available():
            device = "mps"
            print("🚀 Using Apple Silicon GPU (MPS backend)")
        elif torch.cuda.is_available():
            device = "cuda"
            print("🚀 Using NVIDIA GPU (CUDA backend)")
        else:
            device = "cpu"
            print("⚠️ Using CPU - training will be slow")
        
        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        
        # Truncation settings to reduce memory usage
        max_input_length = 512
        max_output_length = 128
        
        # Load and process dataset
        try:
            dataset_samsum_pt = load_from_disk(self.config.data_path)
            print(f"✅ Loaded dataset from {self.config.data_path}")
        except Exception as e:
            print(f"❌ Failed to load dataset: {str(e)}")
            raise
        
        def truncate_example(example):
            return {
                'input_ids': example['input_ids'][:max_input_length],
                'attention_mask': example['attention_mask'][:max_input_length],
                'labels': example['labels'][:max_output_length]
            }
        
        # Select smaller subsets for training and validation
        train_dataset = dataset_samsum_pt["train"].select(range(50)).map(truncate_example)
        eval_dataset = dataset_samsum_pt["validation"].select(range(20)).map(truncate_example)
        print(f"📊 Training on {len(train_dataset)} samples, validating on {len(eval_dataset)} samples")

        # Use dynamic padding to optimize memory usage
        seq2seq_data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model_pegasus,
            padding='longest'
        )

        # Configure TrainingArguments - disable all mixed precision for MPS
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=1,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            gradient_checkpointing=False,  # Disabled for MPS compatibility
            fp16=False,  # Disabled for MPS
            bf16=False,  # Disabled for MPS
            optim="adamw_torch",  # Recommended optimizer
            report_to="none",     # Disable external logging
            no_cuda=(device != "cuda")  # Explicitly disable CUDA if not using it
        )

        print("⚙️ Training arguments configured:")
        print(f"• Device: {device}")
        print(f"• Batch size: {self.config.per_device_train_batch_size}")
        print(f"• Epochs: {self.config.num_train_epochs}")
        print(f"• Precision: full (fp32)")

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )
        
        print("🏁 Starting training...")
        try:
            trainer.train()
            print("🎉 Training completed successfully!")
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print("💥 Out of memory error! Try these solutions:")
                print("1. Reduce max_input_length (current: 512)")
                print("2. Reduce batch size (current: {self.config.per_device_train_batch_size})")
                print("3. Reduce dataset subset sizes (current: train 50, eval 20)")
                print("4. Use gradient checkpointing by setting gradient_checkpointing=True")
            raise
        
        # Save trained model and tokenizer
        output_dir = os.path.join(self.config.root_dir, "pegasus-samsum-model")
        try:
            model_pegasus.save_pretrained(output_dir)
            tokenizer.save_pretrained(output_dir)
            print(f"💾 Model saved to {output_dir}")
        except Exception as e:
            print(f"❌ Failed to save model: {str(e)}")
            raise

# Training pipeline
if __name__ == "__main__":
    try:
        print("🔍 Initializing configuration...")
        config_manager = ConfigurationManager()
        model_trainer_config = config_manager.get_model_trainer_config()
        
        print("🧠 Initializing model trainer...")
        model_trainer = ModelTrainer(config=model_trainer_config)
        
        print("🚀 Starting training process...")
        model_trainer.train()
        
    except Exception as e:
        print(f"⛔ Critical error: {str(e)}")
        print("💡 Troubleshooting tips:")
        print("- Check available memory with `free -h` or Activity Monitor")
        print("- Reduce model size or dataset size")
        print("- Update PyTorch and Transformers: `pip install --upgrade torch transformers`")
        print("- Try CPU fallback: set `device = 'cpu'`")
        raise e

🔍 Initializing configuration...
[2025-08-18 11:26:04,320: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-18 11:26:04,322: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-18 11:26:04,323: INFO: common: created directory at: artifacts]
[2025-08-18 11:26:04,323: INFO: common: created directory at: artifacts/model_trainer]
🧠 Initializing model trainer...
🚀 Starting training process...
🚀 Using Apple Silicon GPU (MPS backend)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Loaded dataset from artifacts/data_transformation/samsum_dataset
📊 Training on 50 samples, validating on 20 samples
⚙️ Training arguments configured:
• Device: mps
• Batch size: 1
• Epochs: 1
• Precision: full (fp32)


/opt/anaconda3/envs/textS/lib/python3.10/site-packages/transformers/training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
/var/folders/7h/q9ssh4wj7r3d99nqq2qzhkb80000gn/T/ipykernel_83925/1939007638.py:137: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🏁 Starting training...


Step,Training Loss,Validation Loss


/opt/anaconda3/envs/textS/lib/python3.10/site-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


🎉 Training completed successfully!
💾 Model saved to artifacts/model_trainer/pegasus-samsum-model
